In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

#Import Libraries
import tensorflow as tf
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

from sklearn.metrics import recall_score, precision_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from models import save_model, generate_model_CNN, generate_model_LSTM, plot_performance
from generators import generator, decode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
save_dir = 'plots'
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

X_train = np.load('../data/final_data/X_train.npy')
y_train = np.load('../data/final_data/y_train.npy')


X_test = np.load('../data/final_data/X_test.npy')
y_test = np.load('../data/final_data/y_test.npy')


X = np.load('../data/final_data/X.npy')
y = np.load('../data/final_data/y.npy')

enc = OneHotEncoder(categories='auto')
y = enc.fit_transform(y.reshape(-1, 1) ,).toarray()
y_train = enc.fit_transform(y_train.reshape(-1, 1) ,).toarray()
y_test = enc.fit_transform(y_test.reshape(-1, 1) ,).toarray()


In [23]:
batch_size = 500
time_window = X_test.shape[1]
n_dim = X_test.shape[2]
predict_early = 0

input_shape = (time_window, n_dim)
class_weight = {0: 1., 1: 1.,}

In [24]:
# input_shape = (time_window, n_dim)
# model = generate_model_CNN(input_shape)

input_shape = (None, n_dim)
model = generate_model_LSTM(input_shape)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 1)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 20)          1760      
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 42        
Total params: 5,082
Trainable params: 5,082
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=50,
                    epochs=15, 
                    verbose=1,
                    validation_data = (X_test, y_test),
#                     class_weight=class_weight,
                    )

save_model(model, name=predict_early)

Train on 1205 samples, validate on 392 samples
Epoch 1/15
1205/1205 [==============================] - 19s 16ms/step - loss: 0.6341 - acc: 0.6664 - val_loss: 0.5409 - val_acc: 0.7908
Epoch 2/15
1205/1205 [==============================] - 17s 14ms/step - loss: 0.6332 - acc: 0.6647 - val_loss: 0.5288 - val_acc: 0.7908
Epoch 3/15
1205/1205 [==============================] - 14s 11ms/step - loss: 0.6290 - acc: 0.6697 - val_loss: 0.5607 - val_acc: 0.7908
Epoch 4/15
1205/1205 [==============================] - 12s 10ms/step - loss: 0.6291 - acc: 0.6689 - val_loss: 0.5547 - val_acc: 0.7908
Epoch 5/15
1205/1205 [==============================] - 13s 11ms/step - loss: 0.6261 - acc: 0.6705 - val_loss: 0.5685 - val_acc: 0.7908
Epoch 6/15
1205/1205 [==============================] - 20s 17ms/step - loss: 0.6288 - acc: 0.6730 - val_loss: 0.5485 - val_acc: 0.7908
Epoch 7/15
1205/1205 [==============================] - 13s 11ms/step - loss: 0.6270 - acc: 0.6697 - val_loss: 0.5570 - val_acc: 0.7908
E

In [ ]:
plot_performance(history)

In [ ]:
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from generators import decode

In [ ]:
res = model.predict(X_test) 

In [ ]:
y_pred = decode(res, threshold=0.5)
y_true = decode(y_test, threshold=0.5)

In [ ]:
print('precision_score: ', np.round(precision_score(y_true, y_pred, average='binary', pos_label=1), decimals=3))
print('recall_score: ', np.round(recall_score(y_true, y_pred, average='binary', pos_label=1), decimals=3))
print('')
print('precision_score: ', np.round(precision_score(y_true, y_pred, average='binary', pos_label=0), decimals=3))
print('recall_score: ', np.round(recall_score(y_true, y_pred, average='binary', pos_label=0), decimals=3))

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print(cm/cm.sum(axis=1)[:, np.newaxis])

In [ ]:
plt.figure()
plt.hist(y_pred[mask])
plt.show()

In [ ]:
mask = y_true == 1



In [ ]:
mask

In [ ]:
res